In [20]:
import requests
import json
from pyvis.network import Network
from math import *

In [21]:
def get_data(symbol: str):
  url = 'https://blue-api.morpho.org/graphql'
  headers = {
      'Accept-Encoding': 'gzip, deflate, br',
      'Content-Type': 'application/json',
      'Accept': 'application/json',
      'Connection': 'keep-alive',
      'DNT': '1',
      'Origin': 'https://blue-api.morpho.org'
  }
  data = json.dumps({
      "query": f"""
      query {{
        vaults(first:100, where: {{assetSymbol_in: ["{symbol}"], chainId_in: [1]}}) {{
          items {{
            symbol
            state {{
              totalAssetsUsd
              allocation {{
                supplyAssetsUsd
                market {{
                  uniqueKey
                  state {{
                    supplyAssetsUsd
                  }}
                }}
              }}
            }}
          }}
        }}
      }}
      """
  })

  response = requests.post(url, headers=headers, data=data)
  response.raise_for_status()
  data = response.json()

  return data

In [22]:
def get_market_symbol(id):
  url = "https://blue-api.morpho.org/graphql"
  query = f"""
    query {{
      marketByUniqueKey(uniqueKey: "{id}") {{
        loanAsset {{
          symbol
        }}
        collateralAsset {{
          symbol
        }}
        lltv
      }}
    }}"""
  r = requests.post(url, json={'query': query})
  r.raise_for_status()
  data = r.json()["data"]["marketByUniqueKey"]


  loan = data["loanAsset"]["symbol"]
  lltv = float(data["lltv"])/1e18
  if data["collateralAsset"] is None:
    return f"Idle {loan}"
  collat = data["collateralAsset"]["symbol"]
  return f"{collat}/{loan} ({lltv})"

In [23]:
data = get_data("DAI")

In [24]:
# Create a PyVis network
net = Network(notebook=True, height="100%", width="100%", bgcolor="#ffffff", font_color="black", directed=True, cdn_resources='in_line')

# Process each vault
for _, vault in enumerate(data['data']['vaults']['items']):
    if 'state' in vault and 'allocation' in vault['state'] and vault['state']['allocation'] != []:
        vault_id = vault['symbol']
        vault_size = vault['state']['totalAssetsUsd']
        net.add_node(vault_id, label=vault_id, title=vault_id, color='#1f78b4', size=(vault_size/1e6), mass=0)
        for allocation in vault['state']['allocation']:
            if 'market' in allocation and 'uniqueKey' in allocation['market']:
                market_id = allocation['market']['uniqueKey']
                market_title = get_market_symbol(market_id)
                market_size = allocation['market']['state']['supplyAssetsUsd']
                if market_size > 0:
                    net.add_node(market_id, label=market_title, title=market_title, color='#33a02c', size=(market_size/1e6))
                    net.add_edge(vault_id, market_id)

# Set the physics layout for the network
net.set_options("""
var options = {
  "nodes": {
    "font": {
      "size": 50,
      "face": "Tahoma",
      "multi": "html"
    },
    "labelThreshold": 20
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -80000,
      "centralGravity": 1,
      "springLength": 100
    },
    "minVelocity": 0.75
  }
}
""")

# Generate network
net.show("bluenet.html")

bluenet.html


In [25]:
from pyvis.network import Network

scale = 1e7

# Create a PyVis network with hierarchical layout
net = Network(notebook=True, height="1200px", width="100%", bgcolor="#ffffff", font_color="black", directed=True, cdn_resources='in_line')

# Process each vault
for _, vault in enumerate(data['data']['vaults']['items']):
    if 'state' in vault and 'allocation' in vault['state'] and vault['state']['allocation'] != []:
        vault_id = vault['symbol']
        vault_size = vault['state']['totalAssetsUsd']
        net.add_node(vault_id, label=vault_id, title=vault_id, color='#1f78b4', size=(vault_size/scale), level=0)
        
        for allocation in vault['state']['allocation']:
            if 'market' in allocation and 'uniqueKey' in allocation['market']:
                market_id = allocation['market']['uniqueKey']
                market_title = get_market_symbol(market_id)
                market_size = allocation['market']['state']['supplyAssetsUsd']
                # Inside the loop where we process each allocation
                if market_size > 0:
                    words = market_title.split()
                    if len(words) > 1:
                        vertical_label = '\n'.join(words)
                    else:
                        vertical_label = '\n'.join([market_title[i:i+3] for i in range(0, len(market_title), 3)])
                    net.add_node(market_id, label=vertical_label, title=market_title, color='#33a02c', 
                                size=(market_size/scale), level=1,
                                font={'size': 10, 'face': 'Tahoma'})
                    
                    # Calculate the width of the edge based on the allocation size
                    allocation_size = allocation['supplyAssetsUsd']

                    if allocation_size > 0:
                        edge_width = allocation_size / scale / 2
                    
                        # Add the edge with the calculated width
                        net.add_edge(vault_id, market_id, width=edge_width, title=f"Allocation: {allocation_size:.2f}")

# Set the physics layout for the network
net.set_options("""
var options = {
    "nodes": {
        "font": {
            "size": 12,
            "face": "Tahoma"
        },
        "shape": "dot"
    },
    "edges": {
        "color": {
            "inherit": true
        },
        "smooth": false,
        "arrows": {
          "to": {
            "enabled": true,
            "scaleFactor": 0.5
          }
        }
    },
    "layout": {
        "hierarchical": {
            "enabled": true,
            "levelSeparation": 300,
            "nodeSpacing": 200,
            "treeSpacing": 200,
            "direction": "UD",
            "sortMethod": "directed"
        }
    },
    "physics": {
        "hierarchicalRepulsion": {
            "centralGravity": 0.8,
            "springLength": 150,
            "springConstant": 0.05,
            "nodeDistance": 250,
            "damping": 0.09
        },
        "minVelocity": 0.75,
        "solver": "hierarchicalRepulsion"
    }
}
""")

# Generate network
net.show("bluenet-layers.html")

bluenet-layers.html
